In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src.utils import ROOT_DIR
import sys
sys.path.append(ROOT_DIR)

from src.dataset import CustomDataset
from src.fcm import FCM
from src.fcm import train_epoch, eval_epoch, acc, f1
import torch

import matplotlib.pyplot as plt
from PIL import Image
import requests
import torch
from src.dataset import CustomDataset
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

In [ ]:
# Load the dataset
DATASET_PATH = os.path.join(ROOT_DIR, 'data', 'MMHS150K', 'MMHS150K_with_img_text.csv')
df = pd.read_csv(DATASET_PATH)

# Load the saved predictions

PREDICTIONS_PATH = os.path.join(ROOT_DIR,'data', 'results','FCM', 'fcm_predictions.json')
OUTPUTS_PATH = os.path.join(ROOT_DIR, 'data','results','FCM', 'fcm_outputs.json')

import json
with open(PREDICTIONS_PATH, 'r') as f:
    predictions = json.load(f)
with open(OUTPUTS_PATH, 'r') as f:
    outputs = json.load(f)
    
# For each index, get the binary_hate and label
pred_keys = [int(k) for k in predictions.keys()]

df_test = df[df["index"].isin(pred_keys)]
df_test["pred"] = [predictions[str(k)] for k in df_test["index"]]
df_test["output"] = [outputs[str(k)] for k in df_test["index"]]

In [ ]:
# Compare "binary_hate" and "pred" to get the accuracy and F1 score
acc_score = acc(df_test["binary_hate"], df_test["pred"])
f1_score = f1(df_test["binary_hate"], df_test["pred"])

print(f"Accuracy: {acc_score}")
print(f"F1 score: {f1_score}")

In [ ]:
# Plot the ROC curve
from sklearn.metrics import roc_curve, auc
import numpy as np

fpr, tpr, _ = roc_curve(df_test["binary_hate"], df_test["output"])
roc_auc = auc(fpr, tpr)

lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve FCM (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

In [ ]:
# Plot the distribution of the outputs / predictions
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].hist(df_test["output"], bins=50)
ax[0].set_title("Distribution of the outputs")
ax[0].set_xlabel("Output")
ax[0].set_ylabel("Count")

ax[1].hist(df_test["pred"], bins=50)
ax[1].set_title("Distribution of the predictions")
ax[1].set_xlabel("Prediction")
ax[1].set_ylabel("Count")

plt.show()

In [ ]:
# Plot the confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(df_test["binary_hate"], df_test["pred"])

print("True positive rate: ", cm[1, 1] / (cm[1, 1] + cm[1, 0]))
print("True negative rate: ", cm[0, 0] / (cm[0, 0] + cm[0, 1]))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
idx = 100
image_index = df_test["index"].values[idx]

img_path = os.path.join(ROOT_DIR, 'data', 'MMHS150K', 'img_resized', str(image_index) + '.jpg')
image = Image.open(img_path)

# Normalize the image between 0 and 1
means_std_path = os.path.join(ROOT_DIR, "data", "MMHS150K", "means_stds.csv")
means_stds = pd.read_csv(means_std_path)
mean = [means_stds["mean_red"][0], means_stds["mean_green"][0], means_stds["mean_blue"][0]]
std = [means_stds["std_red"][0], means_stds["std_green"][0], means_stds["std_blue"][0]]
transform = transforms.Compose(
    [
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=mean,
            std=std,
        ),
    ]
)
image_norm1 = transform(image)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose(
    [
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=mean,
            std=std,
        ),
    ]
)

img_norm2 = transform(image)

# Plot the distributions of values for the three images (image, image_norm1, image_norm2)
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].hist(np.array(image).flatten(), bins=50)
ax[0].set_title("Distribution of the image values")
ax[0].set_xlabel("Value")
ax[0].set_ylabel("Count")

ax[1].hist(image_norm1.flatten(), bins=50)
ax[1].set_title("Distribution of the image values (normalized between 0 and 1)")
ax[1].set_xlabel("Value")
ax[1].set_ylabel("Count")

ax[2].hist(img_norm2.flatten(), bins=50)
ax[2].set_title("Distribution of the image values (normalized with mean and std)")
ax[2].set_xlabel("Value")
ax[2].set_ylabel("Count")

plt.show()